In [4]:
#https://github.com/Lednik7/CLIP-ONNX

In [5]:
# !pip install git+https://github.com/Lednik7/CLIP-ONNX.git
# !pip install git+https://github.com/openai/CLIP.git
# !pip install onnxruntime-gpu

## Export CLIP embeddings to ONNX


In [2]:
#image only
# import torch
# import clip
# from PIL import Image

# # 1) Load your CLIP model
# device = "cpu"
# clip_model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
# clip_model.eval()

# # 2) Define a wrapper module that returns image embeddings
# class CLIPImageEncoder(torch.nn.Module):
#     def __init__(self, clip_model):
#         super().__init__()
#         self.clip_model = clip_model

#     def forward(self, image):
#         # CLIP’s encode_image returns an unnormalized embedding of size [batch_size, 512]
#         return self.clip_model.encode_image(image)

# image_encoder = CLIPImageEncoder(clip_model).to(device)

# # 3) Prepare a dummy input for shape tracing
# dummy_image = preprocess(Image.open("CLIP.png")).unsqueeze(0).to(device)  # [1, 3, 224, 224]

# # 4) Export the custom image-encoder to ONNX
# torch.onnx.export(
#     image_encoder,
#     dummy_image,                       # Only one input
#     "clip_image_encoder.onnx",
#     export_params=True,
#     opset_version=14,                 # for scaled_dot_product_attention
#     do_constant_folding=True,
#     input_names=["image_input"],      # single input
#     output_names=["image_features"],  # single output
#     dynamic_axes={
#         "image_input": {0: "batch_size"},
#         "image_features": {0: "batch_size"},
#     },
# )
# print("Exported CLIP image-encoder to clip_image_encoder.onnx")

/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Exported CLIP image-encoder to clip_image_encoder.onnx


In [1]:
import torch
import clip
from PIL import Image

# 1) Load your CLIP model
device = "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
clip_model.eval()

# 2) Create a wrapper that returns both image and text embeddings
class CLIPEncoder(torch.nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.clip_model = clip_model

    def forward(self, image, text):
        # image:  [batch_size_img, 3, 224, 224]
        # text:   [batch_size_txt, 77]
        image_embeds = self.clip_model.encode_image(image)   # shape [batch_size_img, 512]
        text_embeds  = self.clip_model.encode_text(text)     # shape [batch_size_txt, 512]
        return image_embeds, text_embeds

# Instantiate our wrapper
clip_encoder = CLIPEncoder(clip_model).to(device)
clip_encoder.eval()

# 3) Dummy inputs for shape tracing
dummy_image = preprocess(Image.open("CLIP.png")).unsqueeze(0).to(device)  # shape [1, 3, 224, 224]
dummy_text  = clip.tokenize(["hello world"]).to(device)                   # shape [1, 77]

# 4) Export to ONNX with two inputs and two outputs
input_names  = ["image_input", "text_input"]
output_names = ["image_features", "text_features"]

torch.onnx.export(
    clip_encoder,
    (dummy_image, dummy_text),       # pass both as a tuple
    "clip_image_text_encoder.onnx",
    export_params=True,
    opset_version=14,                # needed for scaled_dot_product_attention
    do_constant_folding=True,
    input_names=input_names,
    output_names=output_names,
    dynamic_axes={
        "image_input":     {0: "batch_size_img"},
        "text_input":      {0: "batch_size_txt"},
        "image_features":  {0: "batch_size_img"},
        "text_features":   {0: "batch_size_txt"},
    },
)
print("Exported CLIP image+text encoder to clip_image_text_encoder.onnx")


/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/opt/conda/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:5385: UserWarning: Exporting aten::index operator of advanced indexing in opset 14 is achieved by combination of multiple ONNX operators, including Reshape, Transpose, Concat, and Gather. If indices include negative values, the exported graph will produce incorrect results.
  warnings.warn(


Exported CLIP image+text encoder to clip_image_text_encoder.onnx


# Text embeddings

In [2]:
# Trying another way as seen here: https://github.com/openai/CLIP/issues/122#issuecomment-1172928859

In [7]:
# ! pip install onnx onnxruntime
# This seems like it works
import clip
import torch

# Change model to ViT-B/32
m, pre = clip.load("ViT-B/32")
npx = m.visual.input_resolution  # Should be 224 for ViT-B/32
dummy_image = torch.randn(10, 3, npx, npx)
dummy_texts = clip.tokenize(["quick brown fox", "lorem ipsum"])
m.forward(dummy_image, dummy_texts)  # Original CLIP result (1)

torch.onnx.export(m, (dummy_image, dummy_texts), "clip_vit.onnx", export_params=True,
  input_names=["IMAGE", "TEXT"],
  output_names=["LOGITS_PER_IMAGE", "LOGITS_PER_TEXT"],
  opset_version=14,
  dynamic_axes={
      "IMAGE": {
          0: "image_batch_size",
      },
      "TEXT": {
          0: "text_batch_size",
      },
      "LOGITS_PER_IMAGE": {
          0: "image_batch_size",
          1: "text_batch_size",
      },
      "LOGITS_PER_TEXT": {
          0: "text_batch_size",
          1: "image_batch_size",
      },
  }
)


# result  # verify that result is comparable to (1)

/opt/conda/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:5385: UserWarning: Exporting aten::index operator of advanced indexing in opset 14 is achieved by combination of multiple ONNX operators, including Reshape, Transpose, Concat, and Gather. If indices include negative values, the exported graph will produce incorrect results.
  warnings.warn(


NameError: name 'result' is not defined

In [ ]:
# Verify with onnxruntime
import onnxruntime as ort

ort_sess = ort.InferenceSession("clip_vit.onnx")
result = ort_sess.run(["LOGITS_PER_IMAGE", "LOGITS_PER_TEXT"], 
  {"IMAGE": dummy_image.numpy(), "TEXT": dummy_texts.numpy()})

In [ ]:
# https://github.com/Lednik7/CLIP-ONNX

In [4]:
!wget -c -O CLIP.png https://github.com/openai/CLIP/blob/main/CLIP.png?raw=true


--2024-12-28 20:20:57--  https://github.com/openai/CLIP/blob/main/CLIP.png?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/openai/CLIP/raw/refs/heads/main/CLIP.png [following]
--2024-12-28 20:20:57--  https://github.com/openai/CLIP/raw/refs/heads/main/CLIP.png
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/openai/CLIP/refs/heads/main/CLIP.png [following]
--2024-12-28 20:20:57--  https://raw.githubusercontent.com/openai/CLIP/refs/heads/main/CLIP.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

 

In [ ]:
import clip
from PIL import Image
import numpy as np

# onnx cannot work with cuda
model, preprocess = clip.load("ViT-B/32", device="cpu", jit=False)

# batch first
image = preprocess(Image.open("CLIP.png")).unsqueeze(0).cpu() # [1, 3, 224, 224]
image_onnx = image.detach().cpu().numpy().astype(np.float32)

# batch first
text = clip.tokenize(["a diagram", "a dog", "a cat"]).cpu() # [3, 77]
text_onnx = text.detach().cpu().numpy().astype(np.int32)

In [ ]:
import clip
from PIL import Image
import numpy as np

# onnx cannot work with cuda
model, preprocess = clip.load("ViT-B/32", device="cpu", jit=False)

# batch first
image = preprocess(Image.open("CLIP.png")).unsqueeze(0).cpu() # [1, 3, 224, 224]
image_onnx = image.detach().cpu().numpy().astype(np.float32)

# batch first
text = clip.tokenize(["a diagram", "a dog", "a cat"]).cpu() # [3, 77]
text_onnx = text.detach().cpu().numpy().astype(np.int32)

In [ ]:
from clip_onnx import clip_onnx

visual_path = "clip_visual.onnx"
textual_path = "clip_textual.onnx"

onnx_model = clip_onnx(model, visual_path=visual_path, textual_path=textual_path)
onnx_model.convert2onnx(image, text, verbose=True)
# ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
onnx_model.start_sessions(providers=["CPUExecutionProvider"]) # cpu mode

In [ ]:
image_features = onnx_model.encode_image(image_onnx)
text_features = onnx_model.encode_text(text_onnx)

logits_per_image, logits_per_text = onnx_model(image_onnx, text_onnx)
probs = logits_per_image.softmax(dim=-1).detach().cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421067 0.00299571]]

In [1]:
# Load saved models:

In [2]:
!wget https://clip-as-service.s3.us-east-2.amazonaws.com/models/onnx/ViT-B-32/visual.onnx
!wget https://clip-as-service.s3.us-east-2.amazonaws.com/models/onnx/ViT-B-32/textual.onnx

--2024-12-28 20:19:37--  https://clip-as-service.s3.us-east-2.amazonaws.com/models/onnx/ViT-B-32/visual.onnx
Resolving clip-as-service.s3.us-east-2.amazonaws.com (clip-as-service.s3.us-east-2.amazonaws.com)... 52.219.98.154, 3.5.132.132, 3.5.130.179, ...
Connecting to clip-as-service.s3.us-east-2.amazonaws.com (clip-as-service.s3.us-east-2.amazonaws.com)|52.219.98.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 351519609 (335M) [application/x-www-form-urlencoded]
Saving to: ‘visual.onnx’

visual.onnx         100%[===================>] 335.23M  16.0MB/s    in 47s     

2024-12-28 20:20:24 (7.17 MB/s) - ‘visual.onnx’ saved [351519609/351519609]

--2024-12-28 20:20:24--  https://clip-as-service.s3.us-east-2.amazonaws.com/models/onnx/ViT-B-32/textual.onnx
Resolving clip-as-service.s3.us-east-2.amazonaws.com (clip-as-service.s3.us-east-2.amazonaws.com)... 3.5.130.26, 52.219.143.10, 52.219.93.74, ...
Connecting to clip-as-service.s3.us-east-2.amazonaws.com (clip

In [5]:
!pip install git+https://github.com/Lednik7/CLIP-ONNX.git
!pip install git+https://github.com/openai/CLIP.git

import clip
from PIL import Image
import numpy as np

from clip_onnx import clip_onnx


  Cloning https://github.com/Lednik7/CLIP-ONNX.git to /tmp/pip-req-build-hcczvaoc
  Running command git clone --filter=blob:none --quiet https://github.com/Lednik7/CLIP-ONNX.git /tmp/pip-req-build-hcczvaoc
  Resolved https://github.com/Lednik7/CLIP-ONNX.git to commit ebd4852b7d3ebf116709abf33b26832acaba947b
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-zbwz92uf
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-zbwz92uf
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


ModuleNotFoundError: No module named 'clip_onnx'

In [6]:
onnx_model = clip_onnx(None)
onnx_model.load_onnx(visual_path="visual.onnx",
                     textual_path="textual.onnx",
                     logit_scale=100.0000) # model.logit_scale.exp()
onnx_model.start_sessions(providers=["CPUExecutionProvider"])

NameError: name 'clip_onnx' is not defined